In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf-8-sig')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
female_word_list = load_doc('female_word_file.txt').split('\n')

In [5]:
male_word_list = load_doc('male_word_file.txt').split('\n')

In [10]:
#yusu wrote this
def ratio_sum(output):
  sum_of_ratio=0
  n=0
  dictionary=corpus.dictionary
  for i in range(0,len(female_word_list)):
    if female_word[i] in dictionary.token2id and male_word[i] in dictionary.token2id:
      female_id = dictionary.token2id[female_word_list[i]] 
      male_id = dictionary.token2id[male_word_list[i]]
      ratio=abs(log(output[female_id]/output[male_id]))
      sum_of_ratio+=ratio
      n+=1
    else:
      i=i+1
  return sum_of_ratio/n

In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i in range(0, data_source.size(0) - 1, args.bptt):
        data, targets = get_batch(data_source, i, evaluation=True)
        output, hidden = model(data, hidden)
        output_flat = output.view(-1, ntokens)
        #yusu added "+"ratio_sum"
        total_loss += (len(data) * criterion(output_flat, targets).data+ratio_sum(output_flat))
        hidden = repackage_hidden(hidden)
    return total_loss[0] / len(data_source)

In [0]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(args.batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        #yusu added ratio_sum...
        loss = (criterion(output.view(-1, ntokens), targets)+ratio_sum(output.view(-1, ntokens)))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.data

        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss[0] / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()